In [1]:
# This notebook is designed for professional data ingestion, cleaning, and feature engineering.

import pandas as pd
import numpy as np
import os

# --- 1. SETUP AND FILE LOADING ---
print("--- Starting Notebook 1: Data Preparation ---")
TRADER_FILE = '/content/historical_data.csv'
SENTIMENT_FILE = '/content/fear_greed_index.csv'
OUTPUT_DIR = 'csv_files'
OUTPUT_FILE_PATH = os.path.join(OUTPUT_DIR, 'daily_metrics_merged.csv')

# Create the required output directory for the standardized submission format
os.makedirs(OUTPUT_DIR, exist_ok=True)

try:
    df_trader = pd.read_csv(TRADER_FILE, low_memory=False)
    df_sentiment = pd.read_csv(SENTIMENT_FILE)
    print(f"Data loaded successfully: Trader Records ({len(df_trader)}), Sentiment Records ({len(df_sentiment)})")
except Exception as e:
    print(f"CRITICAL ERROR: Failed to load data. Ensure '{TRADER_FILE}' and '{SENTIMENT_FILE}' are available. Error: {e}")
    exit()

# --- 2. DATA CLEANING AND PREPROCESSING ---

# 2.1. Clean Trader Data (Historical Data)
print("\n[STEP 2.1] Cleaning Trader Data...")

# Convert Unix timestamp (high precision in ms or similar) to a simple date object
df_trader['Date'] = pd.to_datetime(df_trader['Timestamp'], unit='ms').dt.normalize().dt.date

# Standardize column names for easier access and check for PnL/Volume
# Note: The assignment specifies 'closedPnL' and 'size', but actual columns are 'Closed PnL' and 'Size USD'.
numeric_cols = ['Closed PnL', 'Size USD']
for col in numeric_cols:
    df_trader[col] = pd.to_numeric(df_trader[col], errors='coerce')
    # Fill any NaNs created by coercion (usually none, but good practice)
    df_trader[col].fillna(0, inplace=True)

# --- CRITICAL HANDLING: Missing 'leverage' column ---
LEVERAGE_COL = 'leverage'
if LEVERAGE_COL not in df_trader.columns:
    # As true leverage cannot be calculated without account equity, we use a proxy.
    # We choose a neutral value (1.0) and use PnL/Volume volatility as a risk proxy later.
    df_trader[LEVERAGE_COL] = 1.0
    print(f"  > WARNING: Missing column '{LEVERAGE_COL}'. Created a default proxy column (1.0).")


# 2.2. Clean Sentiment Data
print("[STEP 2.2] Cleaning Sentiment Data...")

# Rename and convert the date column for merging
df_sentiment.rename(columns={'classification': 'Market_Sentiment'}, inplace=True)
df_sentiment['Date'] = pd.to_datetime(df_sentiment['date']).dt.date
# Drop duplicates (if any day has multiple entries) and keep the last (most recent)
df_sentiment = df_sentiment[['Date', 'Market_Sentiment']].drop_duplicates(subset=['Date'], keep='last')
print(f"  > Cleaned Sentiment Data unique days: {len(df_sentiment)}")


# --- 3. FEATURE ENGINEERING & AGGREGATION ---
print("\n[STEP 3] Feature Engineering and Daily Aggregation...")

# Advanced Feature: Volume-Weighted PnL (VW_PnL)
# This metric prioritizes PnL from trades with high conviction (large volume).
df_trader['VW_PnL'] = df_trader['Closed PnL'] * df_trader['Size USD']

# Aggregate Trader Performance Metrics by Date
daily_trader_metrics = df_trader.groupby('Date').agg(
    # --- PROFITABILITY ---
    Avg_Daily_PnL=('Closed PnL', 'mean'), # Measures average trade profitability
    Total_Daily_PnL=('Closed PnL', 'sum'), # Measures total daily profitability
    Median_Daily_PnL=('Closed PnL', 'median'), # Measures typical profitability (less sensitive to large outliers)
    Avg_VW_PnL=('VW_PnL', 'sum'), # Measures conviction-adjusted profitability (Sum of PnL * Volume)

    # --- RISK & VOLATILITY ---
    StdDev_Daily_PnL=('Closed PnL', 'std'), # Measures volatility of trade outcomes (a key risk proxy)
    Avg_Daily_Leverage=(LEVERAGE_COL, 'mean'), # Leverage proxy

    # --- VOLUME & ACTIVITY ---
    Total_Daily_Volume=('Size USD', 'sum'),
    Avg_Daily_Volume=('Size USD', 'mean'),
    Trade_Count=('Account', 'size')
).reset_index()

# Handle potential NaN from single-trade days in StdDev
daily_trader_metrics['StdDev_Daily_PnL'].fillna(0, inplace=True)

print(f"  > Aggregated metrics created: {len(daily_trader_metrics)} days.")


# --- 4. MERGE DATASETS ---
print("\n[STEP 4] Merging Trader Metrics with Market Sentiment...")

df_merged = pd.merge(daily_trader_metrics, df_sentiment, on='Date', how='inner')

print(f"  > Final Merged Dataset size: {len(df_merged)} records (days with both data sources).")
print(f"  > Data range: {df_merged['Date'].min()} to {df_merged['Date'].max()}")

# --- 5. SAVE INTERMEDIATE OUTPUT ---
print("\n[STEP 5] Saving Intermediate Output...")

df_merged.to_csv(OUTPUT_FILE_PATH, index=False)
print(f"SUCCESS: Intermediate merged data saved to: {OUTPUT_FILE_PATH}")
print("--- Notebook 1 Completed. Proceed to Notebook 2 for Analysis. ---")

--- Starting Notebook 1: Data Preparation ---
Data loaded successfully: Trader Records (211224), Sentiment Records (2644)

[STEP 2.1] Cleaning Trader Data...
  > WARNING: Missing column 'leverage'. Created a default proxy column (1.0).
[STEP 2.2] Cleaning Sentiment Data...
  > Cleaned Sentiment Data unique days: 2644

[STEP 3] Feature Engineering and Daily Aggregation...
  > Aggregated metrics created: 7 days.

[STEP 4] Merging Trader Metrics with Market Sentiment...
  > Final Merged Dataset size: 6 records (days with both data sources).
  > Data range: 2023-03-28 to 2025-02-19

[STEP 5] Saving Intermediate Output...
SUCCESS: Intermediate merged data saved to: csv_files/daily_metrics_merged.csv
--- Notebook 1 Completed. Proceed to Notebook 2 for Analysis. ---


/tmp/ipython-input-4140778641.py:40: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_trader[col].fillna(0, inplace=True)
/tmp/ipython-input-4140778641.py:88: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.